In [ ]:
# Our stuff:
import casclik as cc
import casadi as cs
from urdf2casadi import converter, numpy_geom, casadi_geom

# Plotting
from matplotlib import pyplot as plt
import common_plots
import time

In [ ]:
urdf_path = "./urdf/lbr_iiwa_14_r820.urdf"
links = ["base_link",
         "base", 
         "link_1", 
         "link_2", 
         "link_3", 
         "link_4", 
         "link_5", 
         "link_6",
         "link_7",
         "tool0"]
fk_dict = converter.from_file(root="base_link", tip="tool0", filename=urdf_path)

In [ ]:
# Setup time and robot_var
t = cs.MX.sym("t")
q = cs.MX.sym("q", len(fk_dict["joint_names"]))
dq = cs.MX.sym("dq", len(fk_dict["joint_names"]))
# Functions for end-effector things (casadi functions of q)
T_fk = fk_dict["T_fk"]
p_fk = cs.Function("p_fk", [t, q], [T_fk(q)[:3, 3]])
R_fk = cs.Function("R_fk", [t, q], [T_fk(q)[:3, :3]])

In [ ]:
# Test transformation matrix:
T0 = T_fk([0]*len(fk_dict["joint_names"]))
print("Distance to zero pos with transformation matrices: "+str(cs.norm_2(T0[:3,3])))
print("p0:"+str(T0[:3,3]))

In [ ]:
# Check the joint limits from the URDF:
q_max = cs.np.array(fk_dict["upper"])
q_min = cs.np.array(fk_dict["lower"])
print("q_min "+str(q_min))
print("q_max "+str(q_max))

# Define a reasonable max joint speed
max_speed = cs.inf# cs.np.pi/5 # rad/s #cs.inf
print("Max speed: "+str(max_speed))

# Let's pretend home (where we start) is such that we start in the box.
iiwa_home = cs.np.array([-1.5, -1.6447247447521742, 1.4830607057020933, -0.9733457906220311, -0.6578997165707934, 0.0, 0.0])
dt = 0.008

In [ ]:
# Define the basic system limits
# Uphold the joint constraints
joint_limits_cnstr = cc.SetConstraint(
    label="Joint_Limits",
    expression = q,
    set_min = q_min,
    set_max = q_max)

# Let's have some speed limit
joint_speed_limits_cnstr = cc.VelocitySetConstraint(
    label="Joint_speed_limits",
    expression = q,
    set_min = -cs.vertcat([max_speed]*q.size()[0]),
    set_max = cs.vertcat([max_speed]*q.size()[0]))

In [ ]:
# Desired trajectory
omega=0.1
circ_des = cs.vertcat(0.5*cs.cos(omega*t),
                      0.5*cs.sin(omega*t),
                      0.3)
path_des = cs.vertcat(0.5*cs.sin(omega*t)*cs.sin(omega*t) + 0.2,
                      0.5*cs.cos(omega*t)+0.25*cs.sin(omega*t),
                      0.5*cs.sin(omega*t)*cs.cos(omega*t) + 0.1)#0.4)
fpath_des = cs.Function("fpath_des",[t],[path_des])

In [ ]:
# Tracking trajectory
path_cnstr = cc.EqualityConstraint(
    label="move_point",
    expression=p_fk(t, q) - path_des,
    priority=len(fk_dict["joint_names"])+4,
    constraint_type="soft",
    gain=0.15
)
path_cnstr.eval = cs.Function("path_eval", [t,q], [cs.norm_2(path_cnstr.expression)])

In [ ]:
circ_cnstr = cc.EqualityConstraint(
    label="move_circ",
    expression=p_fk(t, q) - circ_des,
    priority=len(fk_dict["joint_names"])+4,
    constraint_type="soft",
    gain=0.15
)
circ_cnstr.eval = cs.Function("circ_eval",[t,q],[cs.norm_2(circ_cnstr.expression)])

In [ ]:
skill_path = cc.SkillSpecification(
    label="skill_path",
    time_var=t,
    robot_var=q,
    robot_vel_var=dq,
    constraints=[path_cnstr, joint_limits_cnstr]
)
skill_circ = cc.SkillSpecification(
    label="skill_circ",
    time_var=t,
    robot_var=q,
    robot_vel_var=dq,
    constraints=[circ_cnstr, joint_limits_cnstr]
)

In [ ]:
# Manipulability
J_p = cs.jacobian(p_fk(t,q+dt*dq), q)
man = cs.mtimes(J_p,J_p.T)
detman = man[0,0]*man[1,1]*man[2,2]+man[0,1]*man[1,2]*man[2,0]+man[0,2]*man[1,0]*man[2,1]
detman += -man[0,2]*man[1,1]*man[2,0] -man[0,1]*man[1,0]*man[2,2] -man[0,0]*man[1,2]*man[2,1]
manipulability_cost = -1e2*detman
for i in range(6):
    manipulability_cost += 1e1*q[i]*q[i]
fmanipulability = cs.Function("fman",[t,q,dq],[manipulability_cost])

In [ ]:
# Let's test all the available controllers
controller_classes = {
    "qp":cc.ReactiveQPController, 
    "nlp":cc.ReactiveNLPController, 
    "pinv":cc.PseudoInverseController, 
    "mpc":cc.ModelPredictiveController
}
controllers = {}
for key in controller_classes.keys():
    controllers[key] = {}

In [ ]:
skill_situations = {
    "path":skill_path,
    "circ":skill_circ
}

In [ ]:
# Compile all the controllers for each situation
for sitn_key in skill_situations.keys():
    print("Compiling skill: "+str(sitn_key))
    for key in controllers.keys():
        t0 = time.time()
        if key == "pinv":
            controllers[key][sitn_key] = controller_classes[key](skill_spec=skill_situations[sitn_key],
                                                                 options={"multidim_sets":True,
                                                                          "damping_factor":1e-16})
        elif key == "mpc":
            controllers[key][sitn_key] = controller_classes[key](skill_spec=skill_situations[sitn_key],
                                                                 cost_expr=cs.dot(dq,dq)+manipulability_cost,
                                                                 horizon_length=5, timestep=dt)
        elif key == "nlp":
            controllers[key][sitn_key] = controller_classes[key](skill_spec=skill_situations[sitn_key],
                                                                 cost_expr=cs.dot(dq,dq)+manipulability_cost)
        else:
            controllers[key][sitn_key] = controller_classes[key](skill_spec=skill_situations[sitn_key])
        controllers[key][sitn_key].setup_problem_functions()
        controllers[key][sitn_key].setup_solver()
        print("\t-"+str(key)+", compile time: "+str(time.time()-t0))

In [ ]:
timesteps = 10000
for cntr_key in controllers.keys():
    print("Simulating controller: "+str(cntr_key))
    for sitn_key in skill_situations.keys():
        print("\t-"+str(sitn_key))
        print("\t\tSetting up initial value problem")
        controllers[cntr_key][sitn_key].setup_initial_problem_solver()
        print("\t\tSolving initial value problem")
        slack_res = controllers[cntr_key][sitn_key].solve_initial_problem(0, iiwa_home)[-1]
        t0 = time.time()
        # Simulate it!
        t_sim = cs.np.array([dt*i for i in range(timesteps+1)])
        t_run_sim = cs.np.array([dt*i for i in range(timesteps)])
        # Robot
        q_sim = cs.np.zeros((len(t_sim),q.shape[0]))
        q_sim[0,:] = iiwa_home
        dq_sim = cs.np.zeros((len(t_sim),q.shape[0]))
        # Cartesian position
        p_sim = cs.np.zeros((len(t_sim), 3))
        p_sim[0,:] = T_fk(iiwa_home)[:3,3].toarray()[:,0]
        # Rotation
        R_sim = cs.np.zeros((len(t_sim), 3, 3))
        R_sim[0,:,:] = T_fk(iiwa_home)[:3,:3].toarray()
        # Error in tracking
        e_sim = cs.np.zeros(len(t_sim))
        e_sim[0] = path_cnstr.eval(t_sim[0],q_sim[0,:])
        # Controller mode
        mode_sim = cs.np.zeros(len(t_sim))
        # Manipulability
        man_sim = cs.np.zeros(len(t_sim))
        man_sim[0] = fmanipulability(0, iiwa_home, [0.]*dq.size()[0])
        # Loop
        for i in range(len(t_sim) - 1):
            t_run0 = time.time()
            res = controllers[cntr_key][sitn_key].solve(t_sim[i],q_sim[i,:],warmstart_slack_var=slack_res)
            t_run_sim[i] = time.time() - t_run0
            dq_sim[i,:] = res[0].toarray()[:,0]
            if res[-1] is not None:
                slack_res = res[-1].toarray()[:,0]
            for idx, dqi in enumerate(dq_sim[i,:]):
                dq_sim[i,idx] = max(min(dqi,max_speed),-max_speed)
            q_sim[i+1,:] = q_sim[i,:] + dq_sim[i,:]*dt
            p_sim[i+1,:] = T_fk(q_sim[i+1,:])[:3,3].toarray()[:,0]
            R_sim[i+1,:,:] = T_fk(q_sim[i+1,:])[:3,:3].toarray()
            e_sim[i+1] = path_cnstr.eval(t_sim[i],q_sim[i+1,:])
            if cntr_key == "pinv":
                mode_sim[i+1] = controllers[cntr_key][sitn_key].current_mode
            man_sim[i+1] = fmanipulability(t_sim[i],q_sim[i,:],dq_sim[i,:])
        controllers[cntr_key][str(sitn_key)+"_res"] = {
            "t_sim":t_sim,
            "t_run_sim": t_run_sim,
            "dq_sim": dq_sim,
            "q_sim": q_sim,
            "p_sim": p_sim,
            "R_sim": R_sim,
            "e_sim": e_sim,
            "mode_sim": mode_sim,
            "man_sim": man_sim
        }
        print("\t\tRuntime: "+str(time.time()-t0))

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots()
for name in controllers.keys():
    ax.plot(controllers[name]["path_res"]["t_sim"],
            controllers[name]["path_res"]["e_sim"],
            label=name)
ax.legend()
ax.set_xlabel("t [s]")
ax.set_ylabel("tracking error [m]")
ax.set_title("path")
#ax.set_yscale("log")

In [ ]:
fig, ax = plt.subplots()
for name in controllers.keys():
    ax.plot(controllers[name]["circ_res"]["t_sim"],
            controllers[name]["circ_res"]["e_sim"],
            label=name)
ax.legend()
ax.set_xlabel("t [s]")
ax.set_ylabel("tracking error [m]")
ax.set_title("circ")
#ax.set_yscale("log")

In [ ]:
ax=common_plots.pos_point_3d(controllers["qp"]["circ_res"])
ax.set_title("QP")
ax=common_plots.pos_point_3d(controllers["nlp"]["circ_res"])
ax.set_title("NLP")
ax=common_plots.pos_point_3d(controllers["mpc"]["circ_res"])
ax.set_title("MPC")
ax=common_plots.pos_point_3d(controllers["pinv"]["circ_res"], p_des=fpath_des)
ax.set_title("PINV")

In [ ]:
ax=common_plots.joints(controllers["qp"]["path_res"])
ax=common_plots.joints(controllers["pinv"]["path_res"])

In [ ]:
fig, ax = plt.subplots()
ax.plot(controllers["pinv"]["path_res"]["t_sim"],controllers["pinv"]["path_res"]["mode_sim"])

In [ ]:
fig, ax = plt.subplots()
cmap = plt.get_cmap("tab10")
for i,key in enumerate(controller_classes.keys()):
    ax.plot(controllers[key]["path_res"]["t_sim"],
            controllers[key]["path_res"]["man_sim"], label=key, color=cmap(i))
ax.legend()

In [ ]:
fig, ax = plt.subplots()
cmap = plt.get_cmap("tab10")
for i,key in enumerate(controller_classes.keys()):
    ax.plot(controllers[key]["circ_res"]["t_sim"],
            controllers[key]["circ_res"]["man_sim"], label=key, color=cmap(i))
ax.legend()

In [ ]:
def ms_format(t):
    return"{:.2f} ms".format(1000.0*t)
cntrllrs_tab = ["pinv","qp","nlp","mpc"]
tab_str = "& PINV & QP & NLPC & MPC\\\\ \n\midrule\n"
tab_str += "Initial (path)"
for cntrl_key in cntrllrs_tab:
    tab_str += "& "+ms_format(controllers[cntrl_key]["path_res"]["t_run_sim"][0])
tab_str += "\\\\\n"
tab_str += "Average (path)"
for cntrl_key in cntrllrs_tab:
    tab_str += "& "+ms_format(cs.np.mean(controllers[cntrl_key]["path_res"]["t_run_sim"]))
print(tab_str)